In [1]:
import os
import time
from pathlib import Path

import globus_sdk
import numpy as np
import pandas as pd
from loguru import logger
from skimage.draw import polygon
from tifffile import imread, imwrite

from multiplex_pipeline.globus_utils import (
    GlobusConfig,
    create_globus_tc,
    get_with_globus_https,
)

In [2]:
df_path = '/Wayne/BLCA/BLCA-6_Analysis/cores.pkl'
input_dir = '/CellDive/BLCA-6/BLCA-6_Final'
temp_dir = r'D:\temp_storage'
output_dir = '/Wayne/BLCA/BLCA-6_Analysis/Core Storage'

selection_list = None

In [3]:
# set up logger

log_file_path = os.path.join(temp_dir, 'core_cutting_log_{time}.log')

# Configure the logger to write to the log file
logger.add(log_file_path, rotation="10 MB", compression="zip")


1

In [4]:
config_dir = Path(r'D:\\globus_config')
gc = GlobusConfig.from_config_files(config_dir)

In [6]:
# create a transfer client
tc = create_globus_tc(gc.client_id, gc.transfer_tokens)

In [7]:
df = pd.read_pickle(get_with_globus_https(df_path, gc.https_server, gc.https_token))
df

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,1152,8768,3072,10624,rectangle,"[[1152.0, 10624.0], [8768.0, 10624.0], [8768.0..."
1,Core_001,1344,9408,12032,20224,rectangle,"[[1344.0, 20224.0], [9408.0, 20224.0], [9408.0..."
2,Core_002,1920,9216,21312,28544,rectangle,"[[1920.0, 28544.0], [9216.0, 28544.0], [9216.0..."
3,Core_003,2944,8896,39296,46464,rectangle,"[[2944.0, 46464.0], [8896.0, 46464.0], [8896.0..."
4,Core_004,2752,9600,48896,56128,rectangle,"[[2752.0, 56128.0], [9600.0, 56128.0], [9600.0..."
...,...,...,...,...,...,...,...
60,Core_060,55360,62144,17984,25280,rectangle,"[[55360.0, 25280.0], [62144.0, 25280.0], [6214..."
61,Core_061,56000,62528,36224,43584,rectangle,"[[56000.0, 43584.0], [62528.0, 43584.0], [6252..."
62,Core_062,56448,64320,46144,54272,rectangle,"[[56448.0, 54272.0], [64320.0, 54272.0], [6432..."
63,Core_063,55808,62528,75456,82496,rectangle,"[[55808.0, 82496.0], [62528.0, 82496.0], [6252..."


In [ ]:
im_name = 'BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F.ome.tif'

task_data = globus_sdk.TransferData(
    source_endpoint=gc.source_collection_id,
    destination_endpoint=gc.destination_collection_id,
    notify_on_succeeded=False,  # Disable email on success
    notify_on_failed=True,      # Enable email on failure
    notify_on_inactive=False    # Disable email on inactivity
)

task_data.add_item(
    input_dir+'/'+im_name,
    './temp.ome.tif')

task_doc = tc.submit_transfer(task_data)
task_id = task_doc["task_id"]
l = logger.info(f"Submitted transfer task {task_id} for {im_name}.")

task = tc.get_task(task_id)
while task["status"] != "SUCCEEDED":
    print('Waiting for the file copy. {time}')
    time.sleep(15)

2025-01-20 16:17:43.760 | INFO     | __main__:<module>:17 - Submitted transfer task fbb2575e-d773-11ef-9be3-191f082cca5d for BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F.ome.tif.


Waiting for the file copy. {time}


KeyboardInterrupt: 

In [ ]:
# keep only the selected cores
if selection_list:
    df = df[df['core_id'].isin(selection_list[0])]


# get the list of images
im_list = [x['name'] for x in tc.operation_ls(gc.source_collection_id, path=input_dir) if ('.ome.tif' in x['name'] and '.0.4_R' in x['name'])]

mask_dict = {}
pending_transfers = []

for im_name in im_list:
    print(im_name)
    im = imread(get_with_globus_https(input_dir+'/'+im_name, gc.https_server, gc.https_token))

    for ind,core in df.iterrows():

        core_im = im[core.row_start:core.row_stop,core.column_start:core.column_stop]

        if core.poly_type != 'rectangle':

            mask = mask_dict.get(core.core_name)

            if mask is None:

                mask = np.zeros_like(core_im, dtype=bool)
                r, c = polygon(core.polygon_vertices[:, 1], core.polygon_vertices[:, 0], mask.shape)

                mask[r, c] = True

                mask_dict[core.core_name] = mask

            core_im[~mask] = 0

        # save the core to a temporary location
        core_name = im_name.replace('.ome.tif',f'_{core.core_name}.ome.tif')
        save_temp_path = os.path.join(temp_dir,core.core_name,core_name)
        os.makedirs(os.path.dirname(save_temp_path), exist_ok=True)
        imwrite(save_temp_path,core_im)

        # transfer the core to the output location
        task_data = globus_sdk.TransferData(
            source_endpoint=gc.destination_collection_id,
            destination_endpoint=gc.source_collection_id,
            notify_on_succeeded=False,  # Disable email on success
            notify_on_failed=True,      # Enable email on failure
            notify_on_inactive=False    # Disable email on inactivity
        )

        task_data.add_item(
            save_temp_path.replace('D:','/~/D').replace('\\','/'),
            output_dir+'/'+core.core_name+'/'+core_name)

        task_doc = tc.submit_transfer(task_data)
        task_id = task_doc["task_id"]
        l = logger.info(f"Submitted transfer task {task_id} for {im_name}.")

        # collect submitted transfer
        pending_transfers.append((task_id, Path(save_temp_path)))

BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F.ome.tif


KeyboardInterrupt: 

In [9]:
def check_transfers(pending_transfers) -> bool:
    """Check status of pending transfers and clean up completed ones."""
    still_pending = []
    all_completed = True

    for task_id, temp_path in pending_transfers:
        task = tc.get_task(task_id)

        if task["status"] == "SUCCEEDED":
            logger.info(f"Transfer {task_id} completed successfully")
            if temp_path.exists():
                temp_path.unlink()
                logger.debug(f"Cleaned up temporary file: {temp_path}")
        elif task["status"] == "FAILED":
            logger.error(f"Transfer {task_id} failed: {task.get('nice_status_details')}")
            all_completed = False
            still_pending.append((task_id, temp_path))
        else:
            all_completed = False
            still_pending.append((task_id, temp_path))

    pending_transfers = still_pending
    return all_completed

In [10]:
while not check_transfers(pending_transfers):
    logger.info("Waiting for transfers to complete...")
    time.sleep(60)  # Check every minute

logger.info("All processing and transfers completed")

2025-01-20 14:09:17.138 | INFO     | __main__:check_transfers:10 - Transfer fbd02b92-d761-11ef-a413-a3a97d3f63df completed successfully
2025-01-20 14:09:17.152 | DEBUG    | __main__:check_transfers:13 - Cleaned up temporary file: D:\temp_storage\Core_000\BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_000.ome.tif
2025-01-20 14:09:17.178 | INFO     | __main__:check_transfers:10 - Transfer fc60bb1c-d761-11ef-a413-a3a97d3f63df completed successfully


2025-01-20 14:09:17.214 | DEBUG    | __main__:check_transfers:13 - Cleaned up temporary file: D:\temp_storage\Core_001\BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_001.ome.tif
2025-01-20 14:09:17.244 | INFO     | __main__:check_transfers:10 - Transfer fcdf82da-d761-11ef-a413-a3a97d3f63df completed successfully
2025-01-20 14:09:17.258 | DEBUG    | __main__:check_transfers:13 - Cleaned up temporary file: D:\temp_storage\Core_002\BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_002.ome.tif
2025-01-20 14:09:17.282 | INFO     | __main__:check_transfers:10 - Transfer fd4da846-d761-11ef-a413-a3a97d3f63df completed successfully
2025-01-20 14:09:17.293 | DEBUG    | __main__:check_transfers:13 - Cleaned up temporary file: D:\temp_storage\Core_003\BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_003.ome.tif
2025-01-20 14:09:17.315 | INFO     | __main__:check_transfers:10 - Transfer fdb4695a-d761-11ef-a413-a3a97d3f63df completed successfully
2025-01-20 14:09:17.333 | DEBUG    | __main__:

In [47]:
pending_transfers

[('77b08bfa-d760-11ef-ace3-d575118df8d3',
  WindowsPath('D:/temp_storage/Core_000/BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_000.ome.tif')),
 ('782fe490-d760-11ef-ace3-d575118df8d3',
  WindowsPath('D:/temp_storage/Core_001/BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_001.ome.tif')),
 ('789e2a04-d760-11ef-ace3-d575118df8d3',
  WindowsPath('D:/temp_storage/Core_002/BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_002.ome.tif')),
 ('78ece4b4-d760-11ef-ace3-d575118df8d3',
  WindowsPath('D:/temp_storage/Core_003/BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_003.ome.tif')),
 ('78ece4b6-d760-11ef-ace3-d575118df8d3',
  WindowsPath('D:/temp_storage/Core_004/BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_004.ome.tif')),
 ('7989fe7a-d760-11ef-ace3-d575118df8d3',
  WindowsPath('D:/temp_storage/Core_005/BLCA-6_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F_Core_005.ome.tif')),
 ('79e36c1c-d760-11ef-ace3-d575118df8d3',
  WindowsPath('D:/temp_storage/Core_006/BLCA-6_1.0.4_R000_Cy3_pH